In [24]:
import json

In [29]:
with open("/mnt/disk1/Alfred/Rescoring/src/RescoreBert/data/tedlium2/noLM/test/token/token.json") as f:
    data = json.load(f)

In [30]:
num_no50 = 0
for d in data:
    if (len(d['token']) != 50):
        num_no50 += 1
        print(f"{d['name']} is {len(d['token'])}")
print(num_no50)

0


In [ ]:
num_no50 = 0
for d in data:
    if (len(d['pll']) != len(d['token'])):
        print(f"illegal:{d['name']}")
    if (len(d['pll']) != 50):
        num_no50 += 1
        print(f"{d['name']} is {len(d['pll'])}")
print(num_no50)